# Загрузка данных и библиотек

In [2075]:
import pandas as pd
import ast
import re

In [2076]:
df = pd.read_csv('main_task.csv')

# Анализ источника данных

In [2077]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


# Предобработка

#### Вспомогательные функции и переменные

In [2078]:
# col_cuisines = 'Cuisine Style'
# col_cuisines_count = 'Cuisines Count'
# col_rest_id = 'Restaurant_id'
# col_price = 'Price Range'
# col_rating = 'Rating'
# col_reviews_count = 'Number of Reviews'
# col_reviews = 'Reviews'
# col_city_reviews = 'Mean City Reviews'
# col_review_days = 'Days Between Reviews'
# col_last_review = 'Last Review Days'
# col_positive = 'Positive Reviews'
RANDOM_STATE = 42

date_pattern = re.compile('\d+/\d+/\d+')
today = pd.to_datetime('today').date()
min_date = pd.to_datetime('1900-01-01').date()

def describe_numerical(col):
    """ Построение гистограммы и вывод основных статистических данных цифровых признаков"""

    df[col].hist(bins=100)
    df[col].describe()

#  вывод основных показателей для категориальных данных
def describe_categorical(col):
    """ Вывод основных статистических данных категориальных признаков"""

    display(df[col].value_counts())
    display(df[col].describe())


def days_since_last_review(reviews):
    """Считает количество дней, прошедших со времени последнего отзыва"""

    review_dates = date_pattern.findall(reviews)
    last_review_date = min_date

    for review_date in review_dates:
        date = pd.to_datetime(review_date).date()

        if date > last_review_date:
            last_review_date = date

    return (today - last_review_date).days


def days_between_reviews(reviews):
    """Считает количество дней, прошедших между отзывами"""

    review_dates = date_pattern.findall(reviews)

    if len(review_dates) == 2:
        date1 = pd.to_datetime(review_dates[0]).date()
        date2 = pd.to_datetime(review_dates[1]).date()

        return abs((date1 - date2).days)

    return 0


def fill_cuisines(cuisines):
    """Преобразует строку, содержащую список в объект списка"""

    return ['Unknown'] if pd.isna(cuisines) else ast.literal_eval(cuisines)


# Анализ данных и создание признаков

## Аттрибут Restaurant_id

#### Анализ данных

In [2079]:
describe_categorical('Restaurant_id')

id_436      18
id_227      18
id_871      18
id_633      18
id_534      17
            ..
id_8021      1
id_8834      1
id_15338     1
id_9626      1
id_11315     1
Name: Restaurant_id, Length: 11909, dtype: int64

count      40000
unique     11909
top       id_436
freq          18
Name: Restaurant_id, dtype: object

#### Создание новых признаков

In [2080]:
# На основе данных анализа данных о Restaurant_id можем сделать вывод о наличии сетевых ресторанов
# и создать новый признак - является ли ресторан сетевым.

# MAE не улучшает

chain_restaurant = df['Restaurant_id'].value_counts().loc[lambda count: count > 1].index

df['Chain Restaurant'] = df['Restaurant_id'].apply(lambda r_id: 1 if r_id in chain_restaurant else 0)


## Аттрибут City

#### Анализ данных

In [2081]:
describe_categorical('City')

London        5757
Paris         4897
Madrid        3108
Barcelona     2734
Berlin        2155
Milan         2133
Rome          2078
Prague        1443
Lisbon        1300
Vienna        1166
Amsterdam     1086
Brussels      1060
Hamburg        949
Munich         893
Lyon           892
Stockholm      820
Budapest       816
Warsaw         727
Dublin         673
Copenhagen     659
Athens         628
Edinburgh      596
Zurich         538
Oporto         513
Geneva         481
Krakow         443
Oslo           385
Helsinki       376
Bratislava     301
Luxembourg     210
Ljubljana      183
Name: City, dtype: int64

count      40000
unique        31
top       London
freq        5757
Name: City, dtype: object

#### Создание новых признаков

###### Является ли город столицей

In [2082]:
capitals = ['Paris','Stockholm','London','Berlin','Bratislava', 'Vienna','Rome',
            'Barcelona','Madrid','Dublin', 'Brussels','Warsaw','Budapest',
            'Copenhagen','Amsterdam','Hamburg','Lisbon','Prague','Oslo','Helsinki',
            'Ljubljana','Athens','Luxembourg']

df['Capital City'] = df['City'].apply(lambda city: 1 if city in capitals else 0)

###### Dummy-variables для городов

In [2083]:
city_ranges = pd.get_dummies(df['City'])

for c in city_ranges.columns:
    df[c] = city_ranges[c].values

In [2084]:


# df[col_cuisines] = df.apply(lambda row: fill_cuisines(row[col_cuisines]), axis=1)
# df[col_review_days] = df.apply(lambda row: days_between_reviews(row[col_reviews]), axis=1)
# df[col_last_review] = df.apply(lambda row: days_since_last_review(row[col_reviews]), axis=1)
# df[col_city_reviews] = df.apply(lambda row: mean_city_reviews[row[col_city]], axis=1)

## Очистка данных

#### Заполняем пропуски

In [2085]:
df['Number of Reviews'] = df['Number of Reviews'].fillna(df['Number of Reviews'].mean())
df['Price Range'] = df['Price Range'].fillna(df['Price Range'].value_counts().index[0])

## Feature Engineering

#### Создаем dummy-признаки для ценовой категории

In [2086]:
# price_ranges = pd.get_dummies(df[col_price])
# price_ranges.columns = ['Inexpensive', 'Middle Price', 'Expensive']
#
# for c in price_ranges.columns:
#     df[c] = price_ranges[c].values


#### Создаем dummy-признаки кухонь и их количества

In [2087]:
# cuisines = set([item for sublist in df[col_cuisines].values for item in sublist])
#
# df[col_cuisines_count] = df.apply(lambda row: len(row[col_cuisines]), axis=1)
#
# for c in cuisines:
#     df[c] = df.apply(lambda row: 1 if c in row[col_cuisines] else 0, axis=1)

#### Создаем dummy-признаки по городам

In [2088]:
# cities = df['City'].unique()
# mean_city_reviews = {}
#
# for city in cities:
#     mean_city_reviews[city] = round(df.query('City == @city')['Number of Reviews'].mean())

# city_ranges = pd.get_dummies(df[col_city])
#
# for c in city_ranges.columns:
#     df[c] = city_ranges[c].values

#### Создаем dummy-признаки по ключевым словам в отзывах

In [2089]:
# review_keywords = ['good', 'delicious', 'best', 'gem', 'excellent', 'amazing', 'great', 'awesome', 'terrible', 'bad', 'worst']
#  return 1 if keyword in review else 0
#
# for keyword in review_keywords:
#     df[keyword] = df.apply(lambda row: parse_review(row[col_reviews], keyword), axis=1)

#### Удаляем столбцы с dtype Object

In [2090]:
object_columns = []

for c in df.columns:
    if(df[c].dtype == 'object' and c != 'Restaurant_id'):
        object_columns.append(c)

df.drop(object_columns, axis=1, inplace=True)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [2091]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Restaurant_id', 'Rating'], axis = 1)
y = df['Rating']

In [2092]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [2093]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=RANDOM_STATE)

# Создаём, обучаем и тестируем модель

In [2094]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [2095]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [2096]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.217187
